# Welcome to my ETL Project

This is a project I did that was inspired by my work. I wanted to make an ETL pipeline for analysis. The question being asked is "Is the cause of a drought influenced by rain
and or soil elements?". There are data sets that can get us this answer. Specifically in Mississippi we experienced a drought in summer of 2023 into winter of 2024.

Now that we have a question lets start digging in to the data.

## About the Data
### Drought Data
Our first dataset we will be using the U.S. Drought Monitor's Drought Severity and Coverage Index (DSCI). Link---> https://droughtmonitor.unl.edu/About/AbouttheData/DSCI.aspx

The data is broken down by counties by state. Our area of interest is Mississippi

Data will be extracted via api and the DSCI is a metric by the week (from Tuedsay-Monday)

### MS city data
Since the drought data's granularity goes to county and our weather datas granularity goes to city, we will extract city and county data. This is the link we will use to join drought data to the weather data. We will scrape the cities that belong to each county in Mississippi and only focus on the highest populated city in each county. Aata is located on wikipedia. we will have to scrape it. Link --> https://en.wikipedia.org/wiki/List_of_municipalities_in_Mississippi

### Weahter data
We will extract weather data from visual crossing via api. link--> https://www.visualcrossing.com/weather-api/

The output is a json. This contains precipitation and all soil data. Since the granularity goes down to city and by date. Since the DSCI is a metric that uses a week, we will aggregate the precipitation by sum and use week and average all our soil fields by the DSCI week.

## About this notebook
The folder prod_code is the production code for how I run the etl pipeline. I thought Jupyter Notebooks is a great way to give you my approach of extracting, transforming, and give you simple examples of the ETL pipeline.

## Final product
After the data is all extracted and transformed we will load it into a duckdb. From there we will extract the duckdb to a CSV and create a tableau dashboard
Link--> https://public.tableau.com/app/profile/jeremy.mccormick/viz/MSDroughtWeatherCorrelationTracker/Dashboard


Now lets get started




###  Extracting drought data

In [4]:
#Importing libraries
import pandas as pd
import requests
from io import StringIO
import datetime as dt

#setting a data variable so that way we can call the API to today whenever we run the ETL pipeline again
today = dt.date.today().strftime('%m/%d/%Y')
#Getting the data
url = f'https://usdmdataservices.unl.edu/api/CountyStatistics/GetDSCI?&aoi=ms&startdate=6/1/2023&enddate={today}&statisticsType=2'
response = requests.get(url)
data = response.text
#This is our dataframe conversion. With response.text it puts the data as a string so with StringIO it reads it in like a file
df1 = pd.read_csv(StringIO(data))
df1.head()


,State,County,FIPS,MapDate,DSCI
0,MS,Adams County,28001,20230530,49
1,MS,Adams County,28001,20230606,49
2,MS,Adams County,28001,20230613,52
3,MS,Adams County,28001,20230620,100
4,MS,Adams County,28001,20230627,100


### Transforming drought data

In [5]:
from datetime import timedelta
#changing data type to date
df1['MapDate'] = pd.to_datetime(df1['MapDate'], format='%Y%m%d')
#Since the DSCI goes from tuesday-monday and we want to aggregate by the week we want the end of the DSCI week. EOW stands for End Of Week
df1['EOW'] = df1['MapDate'] + timedelta(days=6)
#Wee need to know the dates in between each DSCI week so we can aggregate weather data by week
df1['Date'] = df1.apply(lambda x: pd.date_range(start=x['MapDate'], end=x['EOW']), axis=1)
#Since we have the start and stops of the week DSCI week we will explode it by the date because we are going to aggregate the weather data by dsci week
df1 = df1.explode('Date').reset_index(drop=True)
#SOW = Start of week EOW = End of Week
df1.rename(columns={'MapDate': 'SOW'}, inplace=True)
df1.head()

,State,County,FIPS,SOW,DSCI,EOW,Date
0,MS,Adams County,28001,2023-05-30,49,2023-06-05,2023-05-30
1,MS,Adams County,28001,2023-05-30,49,2023-06-05,2023-05-31
2,MS,Adams County,28001,2023-05-30,49,2023-06-05,2023-06-01
3,MS,Adams County,28001,2023-05-30,49,2023-06-05,2023-06-02
4,MS,Adams County,28001,2023-05-30,49,2023-06-05,2023-06-03


### Excracting MS city and county data

In [7]:
#importing libraries
from bs4 import BeautifulSoup
#getting request to the data
url = 'https://en.wikipedia.org/wiki/List_of_municipalities_in_Mississippi'

response = requests.get(url)
#using beautiful to parse throug the html
soup = BeautifulSoup(response.text, 'html.parser')
city_pop_data = soup.find('table')
#converting html into a dataframe
df2 = pd.read_html(str(city_pop_data), header=[0])[0].reset_index(drop=True)
df2.head()

C:\Users\jmcco23a\AppData\Local\Temp\ipykernel_17284\2770064754.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df2 = pd.read_html(str(city_pop_data), header=[0])[0].reset_index(drop=True)


,Name,Type[1],County[1],Population (2020)[1],Population (2010)[1],Change,Land area (2020)[1],Land area (2020)[1].1,Population density,Incorporation date[3]
0,Name,Type[1],County[1],Population (2020)[1],Population (2010)[1],Change,sq mi,km2,Population density,Incorporation date[3]
1,Abbeville,Town,Lafayette,372,419,−11.2%,3.46,9.0,107.5/sq mi (41.5/km2),"June 24, 1870"
2,Aberdeen†,City,Monroe,4961,5612,−11.6%,12.09,31.3,410.3/sq mi (158.4/km2),"May 12, 1837"
3,Ackerman†,Town,Choctaw,1594,1510,+5.6%,2.25,5.8,708.4/sq mi (273.5/km2),"February 16, 1884"
4,Algoma,Town,Pontotoc,705,590,+19.5%,6.60,17.1,106.8/sq mi (41.2/km2),"September 14, 1987"


### Transforming MS City and County data

In [8]:
# Web data is really dirty so I am ending up having to clean this data a lot

#first record looks like headers so i need to drop it
df2 = df2.iloc[1:]
# The city of Jackson has a lot of funky characters and nothing was working so I just replaced it entirely
df2['Name'] = df2['Name'].apply(lambda x: 'Jackson' if 'Jackson' in x else x)
# Removing more funky characters
df2['Name'] = df2['Name'].str.replace('†', '', regex=False).str.replace(r'\[.*?\]', '', regex=True).str.replace('â€¡', '', regex=False).str.strip()
df2[df2['Name']=='Jackson']
#If you notice jackson it has 3 counties associated with it. We need clean that and make jackson be associated with all 3 counties in 3 records

,Name,Type[1],County[1],Population (2020)[1],Population (2010)[1],Change,Land area (2020)[1],Land area (2020)[1].1,Population density,Incorporation date[3]
127,Jackson,City,"Hinds, Madison, Rankin",153701,173514,−11.4%,111.05,287.6,"1,384.1/sq mi (534.4/km2)","February 22, 1840"


In [9]:
#we will split it by "," then explode it then add the word county to it so we can join it with DSCI data.
df2['County[1]'] = df2['County[1]'].str.split(', ')
df2 = df2.explode('County[1]')
#adding string "county" to look it up with the drought data
df2['County[1]'] = df2['County[1]'] + " County"
# Keeping columns I want
df2 = df2[['Name', 'County[1]', 'Population (2020)[1]']]
#Renaming my columns
df2.rename(columns={'Population (2020)[1]': 'Population_2020', 'Name': 'City', 'County[1]': 'County'}, inplace=True)
#Lets have a look now
df2[df2['City']=='Jackson']

,City,County,Population_2020
127,Jackson,Hinds County,153701
127,Jackson,Madison County,153701
127,Jackson,Rankin County,153701


In [10]:

#Changing Data Type to int to get the max population. Some Cities in this state touch multiple counties so a few cities will be duplicated, mainly jackson but we will leave it as is because jackson is a metro. It wouldnt to to much to the DSCI.
df2['Population_2020'] = df2['Population_2020'].astype(int)
#Aggregating it to the get the biggest cities by county
df3 = df2.groupby('County')['Population_2020'].max()
 #Merging the cities together to see what the biggest cities are the biggest in the county
df4 = pd.merge(df3, df2, on=['County', 'Population_2020'], how='left')
# Drop the record with the name 'Total'
df4 = df4[df4['City'] != 'Total']
## Now we have the highest populated city by each county in MS
df4.head()

,County,Population_2020,City
0,Adams County,14520,Natchez
1,Alcorn County,14622,Corinth
2,Amite County,1258,Centreville
3,Attala County,7114,Kosciusko
4,Benton County,551,Ashland


### Extracting Weather Data

In [41]:
#importing libraries
from config import api_key

#this is just an example. I am not going to loop through the whole list of cities in the dataset
city_list = ['Jackson,MS']
 #creating an empty list for my forloop
weather_data = []
#Interating over city
for city in city_list:
    today = dt.date.today().strftime('%Y-%m-%d')
    #this is the api im connecting too.
    url = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city}/2023-06-01/{today}?unitGroup=us&elements=datetime,latitude,longitude,precip,stations,source,soiltemp10,soiltemp20,soilmoisture10,soilmoisture20,soilmoisturevol10,soilmoisturevol20&key={api_key}&contentType=json'
    response = requests.get(url, verify=True)
    data = response.json()
    #getting nesscacary columns i need for the json and calling on the "days" key
    for day in data['days']:
        day_data = {
            'City': city,
            'Date': day['datetime'],
            'Precipitation': day.get('precip', None),
            'SoilTemp10': day.get('soiltemp10', None),
            'SoilTemp20': day.get('soiltemp20', None),
            'SoilMoisture10': day.get('soilmoisture10', None),
            'SoilMoisture20': day.get('soilmoisture20', None),
            'SoilMoistureVol10': day.get('soilmoisturevol10', None),
            'SoilMoistureVol20': day.get('soilmoisturevol20', None)
        }
        weather_data.append(day_data)
df5 = pd.DataFrame(weather_data)
#changing to datetime to merge one
df5['Date'] = pd.to_datetime(df5['Date'])
#cleaning this to join on city
df5['City'] = df5['City'].str.replace(',MS', '').str.strip()
df5.head()
#there is nothing to transform here but we will join everything now down below


,City,Date,Precipitation,SoilTemp10,SoilTemp20,SoilMoisture10,SoilMoisture20,SoilMoistureVol10,SoilMoistureVol20
0,Jackson,2023-06-01,0.000,63.9,63.9,4.2,11.7,0.268,0.296
1,Jackson,2023-06-02,0.000,64.3,64.3,4.2,11.6,0.268,0.295
2,Jackson,2023-06-03,0.045,64.1,64.1,4.3,11.7,0.270,0.298
3,Jackson,2023-06-04,0.000,64.3,64.3,4.1,11.6,0.262,0.294
4,Jackson,2023-06-05,0.326,64.3,64.3,4.1,11.5,0.261,0.293


### Joining all the data together. Well will only see the output of Jackson, MS

In [42]:
df = pd.merge(df1, df2, how='inner', on='County')
df = df.merge(df5, how='inner', on=['City','Date'])
df.head()

,State,County,FIPS,SOW,DSCI,EOW,Date,City,Population_2020,Precipitation,SoilTemp10,SoilTemp20,SoilMoisture10,SoilMoisture20,SoilMoistureVol10,SoilMoistureVol20
0,MS,Hinds County,28049,2023-05-30,0,2023-06-05,2023-06-01,Jackson,153701,0.000,63.9,63.9,4.2,11.7,0.268,0.296
1,MS,Hinds County,28049,2023-05-30,0,2023-06-05,2023-06-02,Jackson,153701,0.000,64.3,64.3,4.2,11.6,0.268,0.295
2,MS,Hinds County,28049,2023-05-30,0,2023-06-05,2023-06-03,Jackson,153701,0.045,64.1,64.1,4.3,11.7,0.270,0.298
3,MS,Hinds County,28049,2023-05-30,0,2023-06-05,2023-06-04,Jackson,153701,0.000,64.3,64.3,4.1,11.6,0.262,0.294
4,MS,Hinds County,28049,2023-05-30,0,2023-06-05,2023-06-05,Jackson,153701,0.326,64.3,64.3,4.1,11.5,0.261,0.293


### Now time for our final transformation of aggregating. This will be our final df that we will use for tableau

In [43]:
#Aggregating the DF by county,city, stating week, and dsci
aggregated_df = df.groupby(['County', 'City', 'SOW', 'DSCI']).agg({
            'Precipitation': 'sum',
            'SoilTemp10': 'mean',
            'SoilTemp20': 'mean',
            'SoilMoisture10': 'mean',
            'SoilMoisture20': 'mean',
            'SoilMoistureVol10': 'mean',
            'SoilMoistureVol20': 'mean'
        }).reset_index()
aggregated_df = aggregated_df.round(2)
aggregated_df.head()

,County,City,SOW,DSCI,Precipitation,SoilTemp10,SoilTemp20,SoilMoisture10,SoilMoisture20,SoilMoistureVol10,SoilMoistureVol20
0,Hinds County,Jackson,2023-05-30,0,0.37,64.18,64.18,4.18,11.62,0.27,0.30
1,Hinds County,Jackson,2023-06-06,0,0.47,64.79,64.79,4.04,11.50,0.26,0.29
2,Hinds County,Jackson,2023-06-13,46,2.75,65.51,65.51,4.03,11.40,0.26,0.29
3,Hinds County,Jackson,2023-06-20,46,0.82,66.29,66.29,4.10,11.46,0.26,0.29
4,Hinds County,Jackson,2023-06-27,46,0.21,67.07,67.07,4.04,11.49,0.26,0.29


# Thanks for checking this project out!

## Please Visit https://public.tableau.com/app/profile/jeremy.mccormick/viz/MSDroughtWeatherCorrelationTracker/Dashboard
